In [27]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, RandomRotation, RandomFlip
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pathlib

In [28]:
train_url = './data/train/'
valid_url = './data/valid/'
test_url = './data/test/'

train_data_dir = pathlib.Path(train_url).with_suffix('')
valid_data_dir = pathlib.Path(test_url).with_suffix('')
test_data_dir = pathlib.Path(valid_url).with_suffix('')

train_image_count = len(list(train_data_dir.glob('*/*.jpg')))
valid_image_count = len(list(valid_data_dir.glob('*/*.jpg')))
test_image_count = len(list(test_data_dir.glob('*/*.jpg')))

print(f'Počet trénovacích obrázků: {train_image_count}')
print(f'Počet validačních obrázků: {valid_image_count}')
print(f'Počet testovacích obrázků: {test_image_count}')

Počet trénovacích obrázků: 4720
Počet validačních obrázků: 165
Počet testovacích obrázků: 165


In [29]:
batch_size = 32
img_height = 32
img_width = 32

In [30]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_data_dir,
    seed=0,
    color_mode='rgb',
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    valid_data_dir,
    seed=0,
    color_mode='rgb',
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 4720 files belonging to 33 classes.
Found 165 files belonging to 33 classes.


In [31]:
class_names = train_ds.class_names
print(class_names)

['archery', 'baseball', 'basketball', 'billiards', 'bmx', 'bowling', 'boxing', 'bull riding', 'cheerleading', 'curling', 'fencing', 'figure skating', 'football', 'formula 1 racing', 'golf', 'high jump', 'hockey', 'horse racing', 'hydroplane racing', 'judo', 'motorcycle racing', 'pole dancing', 'rugby', 'ski jumping', 'snow boarding', 'speed skating', 'surfing', 'swimming', 'table tennis', 'tennis', 'track bicycle', 'volleyball', 'weightlifting']


In [32]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [33]:
normalization_layer = layers.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]        

print(np.min(first_image), np.max(first_image))

0.0 1.0


Model

In [68]:
classes_len = len(class_names)
print(classes_len)
model = Sequential()
model.add(RandomFlip("vertical", input_shape=(img_height, img_width, 3)))
model.add(RandomRotation(0.2))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(classes_len, activation = 'softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

33


In [69]:
model.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_24 (RandomFlip)  (None, 32, 32, 3)        0         
                                                                 
 random_rotation_13 (RandomR  (None, 32, 32, 3)        0         
 otation)                                                        
                                                                 
 conv2d_66 (Conv2D)          (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_65 (MaxPoolin  (None, 15, 15, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_67 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_66 (MaxPoolin  (None, 6, 6, 64)       

In [70]:
epochs = 50
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
)

Epoch 1/50


148/148 [==============================] - 14s 76ms/step - loss: 5.0448 - accuracy: 0.0409 - val_loss: 3.4853 - val_accuracy: 0.0545
Epoch 2/50
148/148 [==============================] - 11s 72ms/step - loss: 3.4742 - accuracy: 0.0581 - val_loss: 3.4518 - val_accuracy: 0.0727
Epoch 3/50
148/148 [==============================] - 11s 73ms/step - loss: 3.4343 - accuracy: 0.0646 - val_loss: 3.4743 - val_accuracy: 0.0545
Epoch 4/50
148/148 [==============================] - 11s 74ms/step - loss: 3.4229 - accuracy: 0.0636 - val_loss: 3.4488 - val_accuracy: 0.0606
Epoch 5/50
148/148 [==============================] - 12s 80ms/step - loss: 3.3967 - accuracy: 0.0714 - val_loss: 3.4577 - val_accuracy: 0.0545
Epoch 6/50
148/148 [==============================] - 11s 76ms/step - loss: 3.3589 - accuracy: 0.0729 - val_loss: 3.3631 - val_accuracy: 0.0848
Epoch 7/50
148/148 [==============================] - 11s 73ms/step - loss: 3.3321 - accuracy: 0.0856 - val_loss: 3.3242 - val_accuracy: 0.0848
Epo

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.preprocessing import image
for i in range(1, 6):
    img_url = './data/test/hockey/' + str(i) + '.jpg'
    img = image.load_img(img_url, target_size=(img_height, img_width))
    imgShow = image.load_img(img_url)

    # Convert the image to an array and expand dimensions
    X = image.img_to_array(img)
    X = np.expand_dims(X, axis=0)
    images = np.vstack([X])

    # Perform inference using the model
    val = model.predict(images, verbose=0)
    index = np.where(val == 1)[1]
    print(val)
    

    predicted_class_index = np.argmax(val)

    # Get the corresponding class label
    predicted_class_label = class_names[predicted_class_index]

    # Join class labels into a comma-separated string
    class_labels_string = ", ".join(class_names)

    # Print the predicted class label and the comma-separated list of all class labels
    print("Predicted class label:", predicted_class_label)
    print((class_names[np.argmax(val)], 100 * np.max(val)))
    # Display the vignette image
    plt.imshow(imgShow)
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(train_ds, val_ds))

In [ ]:
import os
import shutil 

def delete_folders_with_name(root_folder, folder_name_to_delete):
    folder_path = os.path.join(root_folder, folder_name_to_delete)
    try:
        shutil.rmtree(folder_path)
        print(f"Deleted folder: {folder_path}")
    except Exception as e:
        print(f"Error deleting folder: {folder_path} - {e}")

root_folder_train = './data/train/'
root_folder_test = './data/test/'
root_folder_valid = './data/valid/'
folder_name_to_delete = 'cricket'

if os.path.exists(root_folder_train):
    #confirm = input(f"Are you sure you want to delete all '{folder_name_to_delete}' folders in {root_folder}? (y/n): ").strip().lower()
    delete_folders_with_name(root_folder_train, folder_name_to_delete)
    delete_folders_with_name(root_folder_test, folder_name_to_delete)
    delete_folders_with_name(root_folder_valid, folder_name_to_delete)
    print(f"All '{folder_name_to_delete}' folders have been deleted.")
else:
    print("The specified root folder does not exist.")